# Asynchronous Programming 

In [91]:
%serialconnect

Found serial ports: /dev/cu.usbmodem14301, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14301 --baud=115200 
Ready.


## What is Asynchronous Programming?

Let's begin by modeling a simple laboratory application consisting of multiple measurement tasks. We'll label each task $(s, t)$,  where $s$ is a unique name for the task, and $t$ is the time required to complete the measurement.

* Step 1. Task (A, 2). This first task A require 2 seconds to complete.
* Step 2. Following A, tasks (B1, 1), (B2, 3), (B3, 2) are not compute intensive and could be performed at the same time, if possible.
* Step 3. Task (C, 1) is started after the prior steps are completed.

We will attempt to simulate the execution of these tasks assuming that no parallelism is possible

In [72]:
import time

def task(name, duration):
    global start
    print(f"Start {name} at {(time.ticks_ms() - start)/1000:.2f}")
    time.sleep(duration)
    print(f"Finish {name} at {(time.ticks_ms() - start)/1000:.2f}")
    return
    
def expt():
    global start
    start = time.ticks_ms()
    task("A", 2)
    task("B1", 1)
    task("B2", 3)
    task("B3", 2)
    task("C", 1)
    return

expt()

Start A at 0.00
Finish A at 2.00
Start B1 at 2.00
Finish B1 at 3.00
Start B2 at 3.01
Finish B2 at 6.01
Start B3 at 6.01
.Finish B3 at 8.01
Start C at 8.01
Finish C at 9.01


### async/await and asyncio.run()

We begin the journey to an asynchronous code by introducing coroutines. A coroutine begins with the ``async`` keyword. A coroutine will be scheduled for execution by a scheduler.

The ``await`` keyword is used to create and invoke an instance of the coroutine. ``await`` suspends execution of the surrounding code and returns control to the event loop until the results of the coroutine are returned. 

One consideration is that ``await`` can only be used inside a coroutine. 

Finally, instances of coroutines can be run using ``asyncio.run()``.

In [92]:
import time
import uasyncio as asyncio

async def task(name, duration):
    global start
    print(f"Start {name} at {(time.ticks_ms() - start)/1000:.2f}")
    time.sleep(duration)
    print(f"Finish {name} at {(time.ticks_ms() - start)/1000:.2f}")
    return
    
async def expt():
    global start
    start = time.ticks_ms()
    await task("A", 2)
    await task("B1", 1)
    await task("B2", 3)
    await task("B3", 2)
    await task("C", 1)
    return

asyncio.run(expt())

Start A at 0.00
Finish A at 2.00
Start B1 at 2.00
Finish B1 at 3.01
Start B2 at 3.01
Finish B2 at 6.01
Start B3 at 6.01
.Finish B3 at 8.01
Start C at 8.01
Finish C at 9.01


### asyncio.sleep() and asyncio.gather()

At this stage there has been no change in execution as a result of introducing ``async`` and ``await``. 

The first change is to replace ``time.sleep(duration)`` with ``await asyncio.sleep(duration)``. The former is blocking, the second is non-blocking which allows event loop to perform other tasks while waiting for the sleep function to return a result.

The second change is use ``await asyncio.gather()`` to start all of the B tasks at once. ``gather()`` returns when all of those coroutines have returned results.

In [99]:
import time
import uasyncio as asyncio

async def task(name, duration):
    global start
    print(f"Start {name} at {(time.ticks_ms() - start)/1000:.2f}")
    await asyncio.sleep(duration)
    print(f"Finish {name} at {(time.ticks_ms() - start)/1000:.2f}")
    return
    
async def expt():
    global start
    start = time.ticks_ms()
    await task("A", 2)
    await asyncio.gather(task("B1", 1), task("B2", 3), task("B3", 2))
    await task("C", 1)
    return

asyncio.run(expt())

Start A at 0.00
Finish A at 2.00
Start B1 at 2.00
Start B2 at 2.01
Start B3 at 2.01
Finish B1 at 3.01
Finish B3 at 4.01
Finish B2 at 5.01
Start C at 5.01
Finish C at 6.01


## Asynchronous Programming for Lab Instrumentation



In [104]:
import machine
import uasyncio as asyncio

async def blink(led, time_ms):
    while True:
        await asyncio.sleep_ms(time_ms)
        led.toggle
        
led1 = machine.Pin(25, machine.Pin.OUT)
led2 = machine.Pin(1, machine.Pin.OUT)

async def main():
    await asyncio.gather(blink(led1, 100), blink(led2, 300))
    
asyncio.run(main())

..................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] device reports readiness to read but returned no data (device disconnected or multiple access on port?)

